In [27]:
import pandas as pd
import requests

In [28]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..TMG_D_USD_CIF_IX.?startPeriod=2002&endPeriod=2007'

In [29]:
df = pd.DataFrame(requests.get(url).json()['CompactData']['DataSet']['Series']).set_index('@REF_AREA')

In [30]:
df = df[df['Obs'].apply(lambda x: isinstance(x, list))] # drops empties

In [31]:
d = {} # temporary dict to save country by country dataframes
for c in df.index.values: # index values are countries (@REF_AREA) as set above
    d[c] = pd.DataFrame(df.loc[c]['Obs']).rename(columns={'@TIME_PERIOD':'date'})
    # Multiply units by unity multiplier value. 
    d[c]['@OBS_VALUE'] = pd.to_numeric(d[c]['@OBS_VALUE'])

In [32]:
prices =  pd.concat(d, axis=0).reset_index().set_index(['level_0','date']).drop('level_1', 1) 

In [33]:
webserv = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/' # the main URL for the JSON rest API
methodCD = 'CompactData/'                                     # CompactData contains only the data 
methodDS = 'DataStructure/' # This method gives info on the country names, units, and indicator names
series = 'IFS'                                              # International Financial Statistics series
freq = 'A'                                                  # Annual
# Set of IFS indicators of interest 
inds = {'x': 'TXG_FOB_USD', 'q': 'EREER_IX', 'mp': 'TMG_D_USD_CIF_IX', 'xp': 'TXG_D_USD_FOB_IX'}  
date = '?startPeriod=2008&endPeriod=2015' 

urlDS = webserv+methodDS+series   # url to access IMF datastructure method API
# Request data from IMF JSON RESTful API URL above. Navigate to the code list:
dataDS = requests.get(urlDS).json()['Structure']['CodeLists']['CodeList']#
df = pd.DataFrame(dataDS[2]['Code']).set_index('@value')               # area names here
area_names = {c : df['Description'].loc[c]['#text'] for c in df.index.values}

In [47]:
#prices['full_name'] = prices.reset_index()['level_0'].map(area_names)
prices.index

MultiIndex(levels=[[u'1C_092', u'1C_440', u'1C_903', u'A10', u'AR', u'AU', u'BE', u'BR', u'CA', u'CO', u'DE', u'DK', u'E1', u'ES', u'F98', u'FI', u'FR', u'GB', u'GR', u'HK', u'HU', u'IE', u'IL', u'IN', u'IS', u'IT', u'JO', u'JP', u'KR', u'LK', u'MA', u'MT', u'MU', u'NL', u'NO', u'NZ', u'OM', u'PH', u'PK', u'PL', u'R16', u'SE', u'SG', u'SN', u'TG', u'TH', u'TR', u'U2', u'US', u'VE', u'W00', u'XR29', u'XR43', u'XS25', u'ZA'], [u'2002', u'2003', u'2004', u'2005', u'2006', u'2007']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23,

In [41]:
area_names

{u'1C_080': u'Export earnings: fuel',
 u'1C_092': u'Export earnings: nonfuel',
 u'1C_355': u'Curacao & St. Maarten',
 u'1C_356': u'Panama Canal Zone',
 u'1C_440': u'Middle East, North Africa, Afghanistan, and Pakistan',
 u'1C_459': u'Yemen, P.D. Rep.',
 u'1C_473': u'Yemen Arab Rep.',
 u'1C_568': u'Ryukyus',
 u'1C_752': u'Central Bank West Africa',
 u'1C_903': u'Emerging and Developing Europe',
 u'1C_904': u'Central and eastern Europe',
 u'1C_969': u'All Participants',
 u'1C_970': u'Other Holders',
 u'1C_977': u'European Monetary Institution',
 u'1C_994': u'EPU/EF',
 u'1C_ALL': u'All Countries and Country Groups',
 u'1C_ALLC': u'All Countries',
 u'1C_ALLG': u'All Country Groups',
 u'1C_All_Countries_Published': u'All Countries Published',
 u'1C_EMU': u'1C_EMU',
 u'1C_NANSA': u'National Accounts (NSA) Countries',
 u'1C_NASA': u'National Accounts (SA) Countries',
 u'1C_NSC': u'Non SRF countries',
 u'1C_SRF': u'SRF Countries',
 u'1C_SRF_less_EMU': u'SRF (excluding EA)',
 u'1C_SRF_plus_EMU'